# Regressão III - Preparação da base de dados

Tratamento preliminar da base.
* Valores nulos e missing
* Renomeação de variáveis
* Separação treino/teste

In [29]:
import os
import pandas as pd
import numpy as np

In [30]:
# carrega o arquivo bruto
arquivo = './input_data/previsao_de_renda_raw.csv'
df = pd.read_csv(arquivo)

In [31]:
# tamanho do arquivo
size_bytes = os.stat(arquivo).st_size
size_mb = size_bytes / (1024 * 1024)

print("Tamanho do arquivo:", np.round(size_mb, 2), "MB")

Tamanho do arquivo: 65.76 MB


In [32]:
df.head()

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,5762,M,S,N,0,Empresário,Superior completo,Casado,Casa,47,16.717808,2.0,11138.14
1,2015-01-01,1279,M,S,N,0,Assalariado,Superior incompleto,Casado,Casa,30,9.600000,2.0,2424.81
2,2015-01-01,14111,M,S,N,0,Assalariado,Superior completo,Casado,Casa,28,8.208219,2.0,13749.66
3,2015-01-01,8189,M,N,S,2,Assalariado,Médio,Casado,Casa,44,1.301370,4.0,2361.84
4,2015-01-01,1830,F,N,S,2,Assalariado,Superior completo,Casado,Casa,33,1.254795,4.0,790.78


In [33]:
df.shape

(750000, 14)

#### Duplicadas

In [34]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(274548, 14)

#### Tipos de variáveis

Checando os tipos de variáveis

In [35]:
df.dtypes

data_ref                  object
index                      int64
sexo                      object
posse_de_veiculo          object
posse_de_imovel           object
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object

In [36]:
print(df.data_ref[0])

2015-01-01


In [37]:
# tipo da variavel data_ref antes da transformacao
print(type(df.data_ref[0]))

<class 'str'>


In [38]:
# transforma a coluna data_ref para o tipo datetime
df.data_ref = pd.to_datetime(df.data_ref, format='%Y-%m-%d')

# tipo da variavel data_ref pos transformacao
df.data_ref.head(5)

0   2015-01-01
1   2015-01-01
2   2015-01-01
3   2015-01-01
4   2015-01-01
Name: data_ref, dtype: datetime64[ns]

Convertendo os valores 'S' e 'N' para valores booleanos nas colunas "posse_veiculo" e "posse_imovel"

In [39]:
df[["posse_de_veiculo", "posse_de_imovel"]] = df[["posse_de_veiculo", "posse_de_imovel"]].replace({'S': True, 'N': False}).astype(bool)

#### Valores nulos e faltantes

In [40]:
df.isna().any().to_frame()

,0
data_ref,False
index,False
sexo,False
posse_de_veiculo,False
posse_de_imovel,False
qtd_filhos,False
tipo_renda,False
educacao,False
estado_civil,False
tipo_residencia,False


In [41]:
df.isnull().any().to_frame()

,0
data_ref,False
index,False
sexo,False
posse_de_veiculo,False
posse_de_imovel,False
qtd_filhos,False
tipo_renda,False
educacao,False
estado_civil,False
tipo_residencia,False


A coluna tempo_emprego possui valores nulos e faltantes.
Vamos operar sobre esses valores.

In [42]:
# nulos
df.tempo_emprego.isnull().value_counts().to_frame()

,count
tempo_emprego,
False,227505
True,47043


In [43]:
# faltantes
df.tempo_emprego.isna().value_counts().to_frame()

,count
tempo_emprego,
False,227505
True,47043


Como a quantidade de valores faltantes e nulos são iguais, vale a pena conferir se são os mesmos indíces, ou valores diferentes.

In [44]:
checagem_index = df.tempo_emprego.isna().index.equals(df.tempo_emprego.isnull().index)
print("Os indices são iguais." if checagem_index else "Os indices são diferentes.")


Os indices são iguais.


Sendo os indices iguais, e como o número de entradas é grande, não queremos perder essa quantidade de dados. Portanto, os valores vazios/nulos serão preenchidos com a média.

In [45]:
# preenchendo com a média
df.tempo_emprego.fillna(df.tempo_emprego.mean(), inplace=True)

In [46]:
# checando se funcionou
df.tempo_emprego.isna().value_counts().to_frame()

,count
tempo_emprego,
False,274548


#### Separando as bases de treino e teste

Os últimos três meses serão destinados a teste.

In [47]:
df.data_ref.min()

Timestamp('2015-01-01 00:00:00')

In [48]:
df.data_ref.max()

Timestamp('2016-03-01 00:00:00')

In [49]:
data_inicio = ('2016-01-01')
data_final = ('2016-03-01')

df_teste = df[(df['data_ref'] >= data_inicio) & (df['data_ref'] <= data_final)]

In [50]:
df_treino = df.drop(df_teste.index)
df_treino.reset_index()

,level_0,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,5762,M,True,False,0,Empresário,Superior completo,Casado,Casa,47,16.717808,2.0,11138.14
1,1,2015-01-01,1279,M,True,False,0,Assalariado,Superior incompleto,Casado,Casa,30,9.600000,2.0,2424.81
2,2,2015-01-01,14111,M,True,False,0,Assalariado,Superior completo,Casado,Casa,28,8.208219,2.0,13749.66
3,3,2015-01-01,8189,M,False,True,2,Assalariado,Médio,Casado,Casa,44,1.301370,4.0,2361.84
4,4,2015-01-01,1830,F,False,True,2,Assalariado,Superior completo,Casado,Casa,33,1.254795,4.0,790.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211148,211148,2015-12-01,14262,F,True,True,2,Assalariado,Médio,Casado,Casa,30,8.046575,4.0,5468.25
211149,211149,2015-12-01,14694,F,False,False,0,Pensionista,Médio,Solteiro,Casa,63,7.767305,1.0,3360.38
211150,211150,2015-12-01,14176,F,True,False,0,Assalariado,Médio,Casado,Casa,56,20.887671,2.0,10621.79
211151,211151,2015-12-01,9025,M,True,True,0,Assalariado,Superior completo,Casado,Casa,43,6.616438,2.0,2667.27


In [51]:
# reset do index do testes
df_teste.reset_index(drop=True, inplace=True)

In [52]:
df_teste

,data_ref,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2016-01-01,12946,F,False,True,0,Pensionista,Médio,Casado,Casa,61,7.767305,2.0,2137.61
1,2016-01-01,8872,F,False,True,2,Servidor público,Superior completo,Casado,Casa,27,9.301370,4.0,13884.35
2,2016-01-01,16279,F,True,True,3,Assalariado,Médio,Casado,Casa,38,10.967123,5.0,11042.47
3,2016-01-01,8158,M,True,True,3,Servidor público,Médio,Casado,Casa,35,5.553425,5.0,1333.82
4,2016-01-01,9356,F,True,False,0,Assalariado,Médio,Viúvo,Casa,63,9.265753,1.0,17633.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63390,2016-03-01,12173,F,True,False,0,Assalariado,Superior completo,Casado,Casa,39,7.104110,2.0,1395.95
63391,2016-03-01,13598,M,True,False,2,Assalariado,Superior completo,Casado,Casa,49,3.016438,4.0,587.93
63392,2016-03-01,6519,F,False,True,0,Pensionista,Superior completo,Viúvo,Casa,63,7.767305,1.0,1678.12
63393,2016-03-01,9144,F,False,True,0,Pensionista,Superior completo,Casado,Casa,65,7.767305,2.0,4084.37


#### Salvando a base pronta para uso

#### Separando em base de treino e testes, para arquivos feather


In [53]:
# salvando

df_teste.to_feather('./input_data/pr_test.feather')
df_treino.to_feather('./input_data/pr_train.feather')

In [58]:
df_teste.shape

(63395, 14)

In [59]:
df_treino.shape

(211153, 14)

#### Criando a base de dados dummies

In [55]:
df_train = df_treino.drop(['data_ref', 'index'], axis=1)
df_test = df_teste.drop(['data_ref', 'index'], axis=1)
# colunas para serem transformadas em dummies
col_dummies=['sexo',
             'tipo_renda', 
             'educacao', 
             'estado_civil', 
             'tipo_residencia',
             'posse_de_veiculo', 
             'posse_de_imovel']

data_train_dummies = pd.get_dummies(data=df_train, columns=col_dummies, drop_first=True)
data_test_dummies = pd.get_dummies(data=df_test, columns=col_dummies, drop_first=True)

# renomeia as colunas para melhor vizualização
data_train_dummies = data_train_dummies.rename(columns=lambda x: x.replace('á', 'a')
                                                                .replace(' ', '_')
                                                                .replace('ç', 'c')
                                                                .replace('ã', 'a')
                                                                .replace('ó', 'o')
                                                                .replace('ú', 'u')
                                                                .replace('é', 'e'))

# renomeia as colunas para melhor vizualização
data_test_dummies = data_test_dummies.rename(columns=lambda x: x.replace('á', 'a')
                                                                .replace(' ', '_')
                                                                .replace('ç', 'c')
                                                                .replace('ã', 'a')
                                                                .replace('ó', 'o')
                                                                .replace('ú', 'u')
                                                                .replace('é', 'e'))



In [62]:
# salvando as variáveis dummies
data_train_dummies.to_feather('./input_data/pr_train_dummies.feather')
data_test_dummies.to_feather('./input_data/pr_test_dummies.feather')

In [63]:
data_train_dummies.shape

(211153, 25)

In [70]:
data_test_dummies.duplicated().value_counts()

False    63388
True         7
Name: count, dtype: int64